In [1]:
# Jupyter options
%load_ext autoreload
%autoreload

# Some basic imports to use
import tensorflow as tf
import numpy as np
import random
from tensorflow import keras
import matplotlib.pyplot as plt

import argparse
import copy

from markov.s3_boto_data_store import S3BotoDataStore, S3BotoDataStoreParameters
from rl_coach.base_parameters import TaskParameters, Frameworks
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps, RunPhase
from rl_coach.utils import short_dynamic_import
import imp

import markov
from markov import utils
import markov.environments
import markov.presets
import os
import sys

# Just set some seeds so we are consistent across runs
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_random_seed(random_seed) # just so it is forward compatible

# Fix any missing access keys for MINIO
os.environ['S3_ENDPOINT_URL'] = 'http://172.17.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'miniokey123'

# Data loading

In [2]:


def add_items_to_dict(target_dict, source_dict):
    updated_task_parameters = copy.copy(source_dict)
    updated_task_parameters.update(target_dict)
    return updated_task_parameters

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d
        
args = {
"checkpoint_save_secs":  60*3,
"local_model_directory": "./checkpoint",
"num_workers": 1,
"model_s3_bucket": "jplbucket",
"model_s3_prefix": "osr",
"aws_region": "us-east-1",
"save_frozen_graph": True,
}
args = objectview(args)

markov_path = imp.find_module("markov")[1]
preset_location = os.path.join(markov_path, "presets", "mars_presets.py")
path_and_module = preset_location + ":graph_manager"
graph_manager = short_dynamic_import(path_and_module, ignore_module_case=True)

task_parameters = TaskParameters(framework_type=Frameworks.tensorflow,
                                     checkpoint_save_secs=args.checkpoint_save_secs)
task_parameters.__dict__['checkpoint_save_dir'] = args.local_model_directory
task_parameters.__dict__['checkpoint_restore_dir'] = args.local_model_directory
task_parameters.__dict__['experiment_path'] = "./checkpoint/expiremental"
task_parameters.__dict__ = add_items_to_dict(task_parameters.__dict__, args.__dict__)

data_store_params_instance = S3BotoDataStoreParameters(bucket_name=args.model_s3_bucket,
                                                       s3_folder=args.model_s3_prefix,
                                                       checkpoint_dir=args.local_model_directory,
                                                       aws_region=args.aws_region)


data_store = S3BotoDataStore(data_store_params_instance)

if args.save_frozen_graph:
    data_store.graph_manager = graph_manager

graph_manager.data_store_params = data_store_params_instance
graph_manager.data_store = data_store
graph_manager.should_stop = False
#start_graph(graph_manager=graph_manager, task_parameters=task_parameters)

# Data Analysis

In [3]:
#print(dir(graph_manager))
graph_manager.create_graph(task_parameters)
#graph_manager.close()
#graph_manager.restore_checkpoint()r



/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Creating graph - name: BasicRLGraphManager
New Martian Gym environment created...
Total Episodic Reward=0.00 Total Episodic Steps=0.00
Step:0.00 Steering:0.000000 R:0.00 DTCP:44.619237 DT:0.038083 CT:3.03 CTCP:0.000000 PSR: 2000.000000 IMU: 4.700281 X, Y: -0.1799696497494137, 0.10538036303127965
Step:1.00 Steering:1.000000 R:4.00 DTCP:44.623406 DT:0.049596 CT:2.99 CTCP:1.000000 PSR: 1999.000000 IMU: 4.700281 X, Y: -0.18408227530452795, 0.10617419717656133
Creating agent - name: agent

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In p




No checkpoint to restore in: ./checkpoint


In [ ]:
#graph_manager.save_checkpoint()
graph_manager.occasionally_save_checkpoint()
for _ in range(100):
    try:
        graph_manager.improve()
    except KeyboardInterrupt:
        raise
    except:
        pass

Starting to improve simple_rl_graph task index 0
Total Episodic Reward=4.00 Total Episodic Steps=1.00
Step:0.00 Steering:0.000000 R:0.00 DTCP:44.620325 DT:0.005114 CT:3.09 CTCP:0.000000 PSR: 2000.000000 IMU: 0.662675 X, Y: -0.18121359219369632, 0.10374033283797367
Step:1.00 Steering:0.000000 R:2.00 DTCP:44.731540 DT:0.491509 CT:3.17 CTCP:0.000000 PSR: 1999.000000 IMU: 6.573892 X, Y: -0.2889299881112775, 0.1466097288914772
Step:2.00 Steering:1.000000 R:2.00 DTCP:44.756162 DT:0.516765 CT:3.43 CTCP:0.000000 PSR: 1998.000000 IMU: 6.573892 X, Y: -0.3136970735579988, 0.14619293450362256
Step:3.00 Steering:-1.000000 R:2.00 DTCP:44.822967 DT:0.583818 CT:3.39 CTCP:0.000000 PSR: 1997.000000 IMU: 6.573892 X, Y: -0.38038726256826466, 0.15052546849655005
Step:4.00 Steering:0.000000 R:2.00 DTCP:44.912202 DT:0.673485 CT:3.45 CTCP:0.000000 PSR: 1996.000000 IMU: 6.573892 X, Y: -0.46996262000473044, 0.1510059726105161
Step:5.00 Steering:0.000000 R:2.00 DTCP:45.010491 DT:0.774473 CT:3.45 CTCP:0.000000 PS

Step:50.00 Steering:-1.000000 R:1.25 DTCP:51.576367 DT:7.482879 CT:1.02 CTCP:0.000000 PSR: 1950.000000 IMU: 7.331532 X, Y: -7.195328118255957, -0.3257873720905562
Step:51.00 Steering:1.000000 R:1.00 DTCP:51.692831 DT:7.601031 CT:0.91 CTCP:0.000000 PSR: 1949.000000 IMU: 7.331532 X, Y: -7.310139596792287, -0.2985493717911941
Step:52.00 Steering:0.000000 R:1.00 DTCP:51.845241 DT:7.755915 CT:0.73 CTCP:0.000000 PSR: 1948.000000 IMU: 7.331532 X, Y: -7.460206787323785, -0.2605511520430266
Step:53.00 Steering:-1.000000 R:1.00 DTCP:51.982573 DT:7.895735 CT:0.64 CTCP:0.000000 PSR: 1947.000000 IMU: 7.331532 X, Y: -7.5953583570014205, -0.22551939570757626
Rover has sustained sideswipe damage
Step:54.00 Steering:1.000000 R:0.00 DTCP:52.131895 DT:8.047561 CT:0.50 CTCP:0.000000 PSR: 1946.000000 IMU: 7.331532 X, Y: -7.742384246561743, -0.18863556735730141
Training - Name: main_level/agent Worker: 0 Episode: 1 Total reward: 95.25 Steps: 54 Training iteration: 0 
Total Episodic Reward=95.25 Total Episod

Step:0.00 Steering:0.000000 R:0.00 DTCP:44.621583 DT:0.008501 CT:2.97 CTCP:0.000000 PSR: 2000.000000 IMU: 1.311459 X, Y: -0.1823894702657254, 0.10467995947900774
Step:1.00 Steering:0.000000 R:2.00 DTCP:44.628539 DT:0.018131 CT:3.16 CTCP:0.000000 PSR: 1999.000000 IMU: 1.311459 X, Y: -0.18914570309937126, 0.10716213771392527
Step:2.00 Steering:-1.000000 R:2.00 DTCP:44.670864 DT:0.060814 CT:3.17 CTCP:0.000000 PSR: 1998.000000 IMU: 4.132479 X, Y: -0.23165238781753378, 0.10714661515093342
Step:3.00 Steering:0.000000 R:2.00 DTCP:44.742393 DT:0.133205 CT:3.22 CTCP:0.000000 PSR: 1997.000000 IMU: 4.132479 X, Y: -0.3038932851203596, 0.10271875500538867
Step:4.00 Steering:1.000000 R:2.00 DTCP:44.840311 DT:0.232502 CT:3.36 CTCP:0.000000 PSR: 1996.000000 IMU: 4.220074 X, Y: -0.4028342509445225, 0.09607893062749075
Step:5.00 Steering:-1.000000 R:2.00 DTCP:44.963158 DT:0.356018 CT:3.36 CTCP:0.000000 PSR: 1995.000000 IMU: 4.220074 X, Y: -0.5262329091568425, 0.09567821203262881
Step:6.00 Steering:-1.00

Step:51.00 Steering:1.000000 R:1.00 DTCP:51.240732 DT:6.680455 CT:0.94 CTCP:0.000000 PSR: 1949.000000 IMU: 7.848420 X, Y: -6.819214738141926, 0.18902532404906666
Step:52.00 Steering:1.000000 R:1.00 DTCP:51.387890 DT:6.827674 CT:0.96 CTCP:0.000000 PSR: 1948.000000 IMU: 7.848420 X, Y: -6.965901318791145, 0.2008010988316981
Step:53.00 Steering:1.000000 R:1.00 DTCP:51.532470 DT:6.973309 CT:0.96 CTCP:0.000000 PSR: 1947.000000 IMU: 7.848420 X, Y: -7.109179304750141, 0.22258316427663954
Step:54.00 Steering:1.000000 R:1.00 DTCP:51.646003 DT:7.087548 CT:0.98 CTCP:0.000000 PSR: 1946.000000 IMU: 7.848420 X, Y: -7.222910702201628, 0.22482373883984286
Step:55.00 Steering:-1.000000 R:1.25 DTCP:51.747304 DT:7.189147 CT:1.02 CTCP:0.000000 PSR: 1945.000000 IMU: 8.909517 X, Y: -7.3233307104506435, 0.23970060237440804
Step:56.00 Steering:-1.000000 R:1.00 DTCP:51.849517 DT:7.291740 CT:0.97 CTCP:0.000000 PSR: 1944.000000 IMU: 8.909517 X, Y: -7.424683946510989, 0.25435038321996617
Step:57.00 Steering:-1.000

Step:41.00 Steering:1.000000 R:1.25 DTCP:50.121717 DT:5.608790 CT:1.32 CTCP:0.000000 PSR: 1959.000000 IMU: 7.862615 X, Y: -5.706325015882711, 0.06842338845019441
Step:42.00 Steering:0.000000 R:1.25 DTCP:50.249384 DT:5.736545 CT:1.32 CTCP:0.000000 PSR: 1958.000000 IMU: 7.862615 X, Y: -5.833733590337039, 0.07679235822325672
Step:43.00 Steering:-1.000000 R:1.25 DTCP:50.377140 DT:5.864673 CT:1.19 CTCP:0.000000 PSR: 1957.000000 IMU: 7.862615 X, Y: -5.961595967434744, 0.08066881836908589
Step:44.00 Steering:0.000000 R:1.25 DTCP:50.503445 DT:5.991292 CT:1.13 CTCP:0.000000 PSR: 1956.000000 IMU: 7.862615 X, Y: -6.0880832470422535, 0.08353908840832955
Step:45.00 Steering:-1.000000 R:1.25 DTCP:50.617481 DT:6.105411 CT:1.07 CTCP:0.000000 PSR: 1955.000000 IMU: 7.862615 X, Y: -6.201971121774566, 0.0899928693227889
Step:46.00 Steering:0.000000 R:1.25 DTCP:50.719990 DT:6.207986 CT:1.05 CTCP:0.000000 PSR: 1954.000000 IMU: 7.862615 X, Y: -6.304270317760614, 0.09672374961887664
Step:47.00 Steering:1.0000

...

Step:73.00 Steering:-1.000000 R:2.50 DTCP:52.058272 DT:7.845034 CT:1.05 CTCP:1.000000 PSR: 1927.000000 IMU: 5.911195 X, Y: -7.613593034058372, 0.497928116263099
Step:74.00 Steering:1.000000 R:2.50 DTCP:52.053467 DT:7.853326 CT:1.04 CTCP:1.000000 PSR: 1926.000000 IMU: 5.911195 X, Y: -7.608659429840105, 0.49921142090003023
Step:75.00 Steering:-1.000000 R:2.50 DTCP:52.050893 DT:7.862245 CT:1.04 CTCP:1.000000 PSR: 1925.000000 IMU: 5.911195 X, Y: -7.605809272076226, 0.5022725265761429
Step:76.00 Steering:0.000000 R:2.50 DTCP:52.046848 DT:7.870585 CT:1.03 CTCP:1.000000 PSR: 1924.000000 IMU: 5.911195 X, Y: -7.601386878249053, 0.5064505259683095
Step:77.00 Steering:1.000000 R:2.50 DTCP:52.040408 DT:7.882803 CT:1.03 CTCP:1.000000 PSR: 1923.000000 IMU: 5.911195 X, Y: -7.5948086647826445, 0.5077535720026906
Step:78.00 Steering:-1.000000 R:1.25 DTCP:52.037678 DT:7.893760 CT:1.04 CTCP:0.000000 PSR: 1922.000000 IMU: 5.911195 X, Y: -7.591495795629386, 0.5143389851140603
Step:79.00 Steering:1.000000 R

Step:124.00 Steering:1.000000 R:1.00 DTCP:51.971482 DT:8.261163 CT:0.92 CTCP:0.000000 PSR: 1876.000000 IMU: 5.911195 X, Y: -7.5191348468240475, 0.5816562226834655
Step:125.00 Steering:-1.000000 R:2.00 DTCP:51.971980 DT:8.266697 CT:0.94 CTCP:1.000000 PSR: 1875.000000 IMU: 5.911195 X, Y: -7.5194639135212284, 0.5835876225998948
Step:126.00 Steering:1.000000 R:2.00 DTCP:51.969745 DT:8.272421 CT:0.94 CTCP:1.000000 PSR: 1874.000000 IMU: 5.911195 X, Y: -7.51750514312192, 0.5803763625575162
Step:127.00 Steering:1.000000 R:2.00 DTCP:51.969070 DT:8.281169 CT:0.93 CTCP:1.000000 PSR: 1873.000000 IMU: 5.911195 X, Y: -7.516468470017276, 0.5844307538460236
Step:128.00 Steering:-1.000000 R:2.00 DTCP:51.970719 DT:8.284788 CT:0.93 CTCP:1.000000 PSR: 1872.000000 IMU: 5.911195 X, Y: -7.518197584840138, 0.5835966814132805
Step:129.00 Steering:0.000000 R:2.00 DTCP:51.973334 DT:8.291917 CT:0.94 CTCP:1.000000 PSR: 1871.000000 IMU: 5.911195 X, Y: -7.520669176636492, 0.5853281875791112
Step:130.00 Steering:0.00

Step:175.00 Steering:-1.000000 R:2.00 DTCP:51.894166 DT:8.796365 CT:0.75 CTCP:1.000000 PSR: 1825.000000 IMU: 5.911195 X, Y: -7.431104925941726, 0.6976395945242586
Step:176.00 Steering:0.000000 R:1.00 DTCP:51.892897 DT:8.810448 CT:0.77 CTCP:0.000000 PSR: 1824.000000 IMU: 5.911195 X, Y: -7.429057660017708, 0.7061354148729407
Step:177.00 Steering:0.000000 R:1.00 DTCP:51.891550 DT:8.828798 CT:0.74 CTCP:0.000000 PSR: 1823.000000 IMU: 5.911195 X, Y: -7.427520713205476, 0.7081601060954584
Step:178.00 Steering:1.000000 R:1.00 DTCP:51.886827 DT:8.851633 CT:0.73 CTCP:0.000000 PSR: 1822.000000 IMU: 5.911195 X, Y: -7.4211367790149785, 0.7261488130844096
Step:179.00 Steering:-1.000000 R:1.00 DTCP:51.888339 DT:8.863314 CT:0.73 CTCP:0.000000 PSR: 1821.000000 IMU: 5.911195 X, Y: -7.423057760475978, 0.7217447730637402
Step:180.00 Steering:0.000000 R:2.00 DTCP:51.894204 DT:8.877091 CT:0.71 CTCP:1.000000 PSR: 1820.000000 IMU: 5.911195 X, Y: -7.4294557070145295, 0.7161739921042669
Step:181.00 Steering:-1.

Step:226.00 Steering:1.000000 R:2.00 DTCP:51.949413 DT:9.479632 CT:0.71 CTCP:1.000000 PSR: 1774.000000 IMU: 8.015176 X, Y: -7.486219058404958, 0.7016123912414314
Step:227.00 Steering:1.000000 R:2.00 DTCP:51.939061 DT:9.494837 CT:0.71 CTCP:1.000000 PSR: 1773.000000 IMU: 8.015176 X, Y: -7.475893449395076, 0.7008488062815161
Step:228.00 Steering:0.000000 R:2.00 DTCP:51.935999 DT:9.502876 CT:0.71 CTCP:1.000000 PSR: 1772.000000 IMU: 8.015176 X, Y: -7.472483421926446, 0.7045377319537816
Step:229.00 Steering:-1.000000 R:2.00 DTCP:51.936185 DT:9.510628 CT:0.71 CTCP:1.000000 PSR: 1771.000000 IMU: 8.015176 X, Y: -7.472757930714932, 0.7035749989675445
Step:230.00 Steering:0.000000 R:2.00 DTCP:51.927324 DT:9.526247 CT:0.70 CTCP:1.000000 PSR: 1770.000000 IMU: 8.015176 X, Y: -7.463126340956366, 0.7116408997323576
Step:231.00 Steering:1.000000 R:1.00 DTCP:51.914139 DT:9.547047 CT:0.69 CTCP:0.000000 PSR: 1769.000000 IMU: 8.015176 X, Y: -7.449073267935088, 0.7205545001836463
Step:232.00 Steering:-1.000

Step:277.00 Steering:-1.000000 R:1.00 DTCP:51.935424 DT:9.953071 CT:0.72 CTCP:0.000000 PSR: 1723.000000 IMU: 8.015176 X, Y: -7.471679053145485, 0.7070319287540199
Step:278.00 Steering:-1.000000 R:2.00 DTCP:51.933928 DT:9.961305 CT:0.72 CTCP:1.000000 PSR: 1722.000000 IMU: 8.015176 X, Y: -7.469955930827457, 0.7094607622519016
Step:279.00 Steering:1.000000 R:2.00 DTCP:51.929205 DT:9.978802 CT:0.71 CTCP:1.000000 PSR: 1721.000000 IMU: 8.015176 X, Y: -7.464030856091439, 0.7224296838164788
Step:280.00 Steering:1.000000 R:2.00 DTCP:51.926185 DT:9.987851 CT:0.71 CTCP:1.000000 PSR: 1720.000000 IMU: 8.015176 X, Y: -7.460702055110698, 0.7256723211275462
Step:281.00 Steering:1.000000 R:1.00 DTCP:51.928345 DT:9.997501 CT:0.72 CTCP:0.000000 PSR: 1719.000000 IMU: 8.015176 X, Y: -7.463135703761336, 0.7227724450127966
Step:282.00 Steering:-1.000000 R:2.00 DTCP:51.927324 DT:10.006638 CT:0.70 CTCP:1.000000 PSR: 1718.000000 IMU: 8.015176 X, Y: -7.462322805919368, 0.7204533070920819


# Create Model

In [ ]:
graph_manager.save_checkpoint()

# Model Performance

# Model Analysis